In [1]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
import pandas as pd
import pylab as pl
import numpy as np
import seaborn as sns
%matplotlib inline

from sklearn import linear_model, preprocessing, pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [3]:
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv('../../data/raw/sdBShortP_large_BPS_set.csv')

# Data preparation

In [6]:
# extract the alpha value from the ce_parameters (alpha_ce is always the same as alpha_th)
df['alpha_ce'] = df['ce_parameters'].apply(lambda x: eval(x)['a_ce'])

In [7]:
# Mark systems with a He ignition or burning error as error. ignore other errors.
df['error'] = df['error_flags'].apply(lambda x: 1 if 4 in eval(x) or 5 in eval(x) else 0)

In [8]:
# remove systems that merge or are contact systems (the latter are likely also mergers)
df = df[(df['stability'] != 'merger') & (df['stability'] != 'contact')]

In [9]:
df['product'] = 'UK'

for i, line in df.iterrows():
    
    sdA = line['sdA']
    sdB = line['sdB']
    sdO = line['sdO']
    HeBurn = line['HeCoreBurning']
    HeWD = line['He-WD']
    
    product = 'failed'
    if line['stability'] == 'CE' or line['stability'] == 'merger' or line['stability'] == 'contact':
        product = 'UK'
    elif sdA:
        product = 'sdA'
    elif sdB:
        product = 'sdB'
    elif sdO:
        product = 'sdO'
    elif HeWD:
        product = 'He-WD'
    elif HeBurn:
        product = 'HB'
    
    df.loc[i, 'product'] = product

In [10]:
df['stability'].value_counts()

stable    6387
CE        3276
Name: stability, dtype: int64

In [11]:
df_ce =df[df['stability'] == 'CE']
df_stable = df[(df['stability'] == 'stable') & (df['error'] == 0)]
df_he = df_stable[df_stable['HeCoreBurning'] == 1]

# Add features

In [ ]:
df_stable['log_M1env_MLend'] = np.log10(df_stable['log_M1env_MLend'])
df_stable['M1core_MLend'] = np.log10(df_stable['M1core_MLend'])
df_stable['log_M1env_MLend'] = np.log10(df_stable['log_M1env_MLend'])
df_stable['M1core_MLend'] = np.log10(df_stable['M1core_MLend'])

In [ ]:
preprocessing.PolynomialFeatures

# Regression Model

In [26]:
X_features = ['M1_init', 'FeH_init', 'M1env_MLend', 'M1core_MLend']
y_features = 'HeCoreBurning'

In [27]:
df_select = df_stable[~df_stable[X_features].isna().any(axis=1)]
df_select[y_features] = df_select[y_features].apply(lambda x: 1 if x else 0)

In [28]:
data_X = df_select[X_features]
data_y = df_select[y_features]
train_X, test_X, train_y, test_y = train_test_split(data_X, data_y, test_size=0.20)

In [32]:
numeric_features = X_features
min_max_features = []
onehot_features = []
ordinal_features = []

preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessing.RobustScaler(), numeric_features),
        ('minmax', preprocessing.MinMaxScaler(), min_max_features),
        ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'), onehot_features),
        ('ordinal', preprocessing.OrdinalEncoder(), ordinal_features),
    ],
    remainder = 'passthrough'
)

train_X_scaled = preprocessor.fit_transform(train_X)
test_X_scaled = preprocessor.transform(test_X)
data_X_scaled = preprocessor.transform(data_X)

In [93]:
pipe = pipeline.Pipeline([#('scaler', preprocessing.StandardScaler()), 
                          ('poly', preprocessing.PolynomialFeatures(2, interaction_only=False)),
                          ('clf', linear_model.LogisticRegression())])

In [94]:
pipe.fit(train_X_scaled, train_y)

Pipeline(steps=[('poly', PolynomialFeatures()), ('clf', LogisticRegression())])

In [95]:
print('Accuracy:')
print('Training score: ', pipe.score(train_X_scaled, train_y))
print('Test score: ', pipe.score(test_X_scaled, test_y))

Accuracy:
Training score:  0.9780434782608696
Test score:  0.9721739130434782


In [96]:
pipe['poly'].powers_

array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [2, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 0, 1, 0],
       [1, 0, 0, 1],
       [0, 2, 0, 0],
       [0, 1, 1, 0],
       [0, 1, 0, 1],
       [0, 0, 2, 0],
       [0, 0, 1, 1],
       [0, 0, 0, 2]])

In [47]:
pipe['clf'].coef_

array([[ 2.74175390e-04,  5.25643806e+00,  1.37381300e-02,
         3.75502362e+00,  1.00386271e+01,  8.51448852e-02,
         2.75923535e+00, -1.11636663e+01, -4.60265930e-01,
         5.48460581e-01,  2.62550988e+00]])

In [79]:
def get_name(powers, feature_names):
    
    name = ""
    for p, f in zip(powers, feature_names):
        if p > 0:
            if name != "":
                name += " * "
            if p > 1:
                name += str(f) + '^' + str(p)
            else:
                name += str(f)
    
    if name == "":
        name = 'bias'
            
    return name
    

def print_coef(pipe, feature_names):
    powers_ = pipe['poly'].powers_
    coef_ = pipe['clf'].coef_
    
    for p, c in zip(powers_, coef_[0]):
        
        feature = get_name(p, feature_names)
        print("{:30s} : {:+0.3f}".format(feature, c))

In [97]:
print_coef(pipe, X_features)

bias                           : -0.170
M1_init                        : +7.070
FeH_init                       : +0.094
M1env_MLend                    : +5.076
M1core_MLend                   : +11.922
M1_init^2                      : +6.160
M1_init * FeH_init             : -1.095
M1_init * M1env_MLend          : +1.813
M1_init * M1core_MLend         : -8.486
FeH_init^2                     : +0.052
FeH_init * M1env_MLend         : -0.415
FeH_init * M1core_MLend        : +0.191
M1env_MLend^2                  : -1.099
M1env_MLend * M1core_MLend     : -0.060
M1core_MLend^2                 : -0.368


In [91]:
pipe['scaler'].__dict__

{'with_mean': True,
 'with_std': True,
 'copy': True,
 'n_features_in_': 4,
 'n_samples_seen_': 4600,
 'mean_': array([-0.01798665, -0.21490626,  0.58101157, -0.19129915]),
 'var_': array([0.34019768, 1.18003123, 1.10313429, 0.6569913 ]),
 'scale_': array([0.58326467, 1.08629242, 1.050302  , 0.81055   ])}